In [1]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import GrubbsTest

In [2]:
def build_sn_legend_handles(sn_keys, palette_dict, linestyles_dict, linewidths_dict=None):
    return_handles = []
    for sn in sn_keys:
        color = palette_dict[sn]
        linestyle = linestyles_dict[sn]
        linewidth = linewidths_dict[sn] if linewidths_dict else 1
        line2d = Line2D([0], [0], label=sn, marker=None, 
                        color=color, linewidth=linewidth)
        return_handles.append(line2d)
    return return_handles

In [3]:
conn_outages = Utilities.get_utldb01p_oracle_connection()
conn_aws = Utilities.get_athena_prod_aws_connection()

In [4]:
fig_num = 0

# ---------------------------------------------------------------
# OUTAGES
# ---------------------------------------------------------------

# NOTE
11946344 is scheduled output.  So, there are apparently no corresponding end events!

In [5]:
# NOTE
# 11946344 is scheduled output.  So, there are apparently no corresponding end events!

# #dates_of_outages = ['2020-10-12']
# outg_rec_nbs = ['11947217', '11946640', '11946344']

# #dates_of_outages = ['2020-10-12', '2017-09-16']
# outg_rec_nbs = ['11947217', '11946640', '11946344', '10143524']

# #dates_of_outages = ['2020-10-12', '2017-09-16']
# outg_rec_nbs = ['11947217', '11946640', '10143524']

# #dates_of_outages = ['2020-07-01']
# outg_rec_nbs = ['11770899']

# #dates_of_outages = ['2020-10-12', '2017-09-16', '2020-07-01']
#outg_rec_nbs = ['11947217', '11946640', '10143524', '11770899']

# outg_rec_nbs = ['11947217', '11946640', '10143524', '11770899', '11751094', '11738944', 
#                 '11496239', '11818003', '11833199', '11851647', '12022118']

outg_rec_nbs = ['12069393', '12051783', '12069529', '12076764', '12058007']
#--------------------------------------------------
sql_outage = DOVSOutages_SQL.build_sql_std_outage(outg_rec_nbs=outg_rec_nbs)
sql_outage.print()

SELECT
	DOV.CI_NB,
	DOV.CMI_NB,
	DOV.OUTG_REC_NB,
	DOV.OUTAGE_NB,
	DOV.DT_ON_TS,
	DOV.DT_OFF_TS,
	DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24) AS DT_OFF_TS_FULL,
	DOV.STEP_DRTN_NB,
	EXTRACT(YEAR FROM DOV.DT_OFF_TS) AS START_YEAR,
	DOV.OPERATING_UNIT_ID,
	DOV.STATE_ABBR_TX,
	DOV.MJR_CAUSE_CD,
	DOV.MNR_CAUSE_CD,
	DOV.LOCATION_ID,
	DOV.GIS_CRCT_NB,
	DOV1.OPRTG_UNT_NM,
	DOV3.DVC_TYP_NM,
	DOV3.SHORT_NM AS SHORT_NM_CLR_DEV,
	DOV4.EQUIP_TYP_NM,
	DOV4.SHORT_NM AS SHORT_NM_EQP_TYP,
	DOV5.MJR_CAUSE_NM,
	DOV5.MNR_CAUSE_NM,
	PRIM.OFF_TM,
	PRIM.REST_TM,
	PRIM.PREMISE_NB
FROM DOVSADM.DOVS_OUTAGE_FACT DOV
	LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
	LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
	LEFT OUTER JOIN DOVSADM.DOVS_CLEARING_DEVIC

In [6]:
dovs_outgs = DOVSOutages(df_construct_type=DFConstructType.kRunSqlQuery, 
                         build_sql_function_kwargs=dict(
                             outg_rec_nbs=outg_rec_nbs
                         ), 
                         init_df_in_constructor=True)

#search_time_window = 60*12 #minutes
search_time_window = 60*72 #minutes
search_time_window = datetime.timedelta(minutes=search_time_window)
dovs_outgs.set_search_time(
    search_time_window,     
    power_out_col='DT_OFF_TS_FULL', power_on_col='DT_ON_TS', 
    t_search_min_col='t_search_min', t_search_max_col='t_search_max'
)

df_outage = dovs_outgs.get_df()

In [7]:
df_outage.shape

(95, 28)

# ---------------------------------------------------------------
# USAGE NONVEE
# ---------------------------------------------------------------

In [8]:
cols_of_interest_usage = TableInfos.AMINonVee_TI.std_columns_of_interest
cols_of_interest_met_prem = TableInfos.MeterPremise_TI.std_columns_of_interest

### BEGIN compare usg_split_to_CTEs = True False

In [9]:
# FUCK
import AMI_SQL
reload(AMI_SQL)
from AMI_SQL import AMI_SQL

import AMINonVee_SQL
reload(AMINonVee_SQL)
from AMINonVee_SQL import AMINonVee_SQL

import GenAn
reload(GenAn)
from GenAn import GenAn


In [10]:
start=time.time()
usg_match_OG = AMINonVee(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function=AMINonVee_SQL.build_sql_usg_for_outages, 
    build_sql_function_kwargs=dict(
        cols_of_interest=cols_of_interest_usage, 
        df_outage=df_outage, 
        split_to_CTEs=False, 
        build_sql_function_kwargs=dict(
            opco='oh', 
            aep_derived_uoms_and_idntfrs=['KWH']
        ), 
        join_mp_args=dict(join_with_CTE=True, 
                          build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem))
    ), 
    init_df_in_constructor=True
)
print('*'*50)
print(f'Time match OG: {time.time()-start}')
print('*'*50)
df_usage_match_OG = usg_match_OG.get_df().copy()
print(f"len(df_usage_match_OG['serialnumber'].unique())     = {len(df_usage_match_OG['serialnumber'].unique())}")
print(f'df_usage_match_OG.shape = {df_usage_match_OG.shape}')

**************************************************
Time match OG: 192.4985728263855
**************************************************
len(df_usage_match_OG['serialnumber'].unique())     = 57
df_usage_match_OG.shape = (89076, 31)


In [11]:
start=time.time()
usg = AMINonVee(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function=AMINonVee_SQL.build_sql_usg_for_outages, 
    build_sql_function_kwargs=dict(
        cols_of_interest=cols_of_interest_usage, 
        df_outage=df_outage, 
        split_to_CTEs=True, 
        build_sql_function_kwargs=dict(
            opco='oh', 
            aep_derived_uoms_and_idntfrs=['KWH']
        ), 
        join_mp_args=dict(join_with_CTE=True, 
                          build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem))
    ), 
    init_df_in_constructor=True
)
print('*'*50)
print(f'Time: {time.time()-start}')
print('*'*50)
df_usage = usg.get_df().copy()
print(f"len(df_usage['serialnumber'].unique())     = {len(df_usage['serialnumber'].unique())}")
print(f'df_usage.shape = {df_usage.shape}')

**************************************************
Time: 74.44975137710571
**************************************************
len(df_usage['serialnumber'].unique())     = 57
df_usage.shape = (89076, 32)


In [12]:
usg.df_equals(usg_match_OG)

True

In [13]:
sort_by = ['aep_endtime_utc', 'serialnumber', 'aep_srvc_qlty_idntfr']
df_usage          = df_usage.sort_values(by=sort_by, ignore_index=True)
df_usage_match_OG = df_usage_match_OG.sort_values(by=sort_by, ignore_index=True)
print(df_usage[df_usage_match_OG.columns].equals(df_usage_match_OG))

True


### END compare usg_split_to_CTEs = True False

In [14]:
start=time.time()
usg = AMINonVee(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function=AMINonVee_SQL.build_sql_usg_for_outages, 
    build_sql_function_kwargs=dict(
        cols_of_interest=cols_of_interest_usage, 
        df_outage=df_outage, 
        split_to_CTEs=True, 
        build_sql_function_kwargs=dict(
            opco='oh', 
            aep_derived_uoms_and_idntfrs=None
        ), 
        join_mp_args=dict(join_with_CTE=True, 
                          build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem))
    ), 
    init_df_in_constructor=True
)
print('*'*50)
print(f'Time match OG: {time.time()-start}')
print('*'*50)
df_usage = usg.get_df().copy()
print(f"len(df_usage['serialnumber'].unique())     = {len(df_usage['serialnumber'].unique())}")
print(f'df_usage.shape = {df_usage.shape}')

**************************************************
Time match OG: 93.99643039703369
**************************************************
len(df_usage['serialnumber'].unique())     = 57
df_usage.shape = (182804, 32)


In [17]:
df_usage = usg.get_df().copy()
df_usage['outg_rec_nb'] = df_usage['OUTG_REC_NB_GPD_FOR_SQL'].astype(int)
df_usage = AMINonVee.perform_std_initiation_and_cleaning(df_usage)
#df_usage = df_usage[df_usage['aep_derived_uom']=='KWH']

In [18]:
print(f'Before combine_kwh_delivered_and_received_values: {df_usage.shape}')
df_usage = AMINonVee.combine_kwh_delivered_and_received_values(df_usage)
print(f'After combine_kwh_delivered_and_received_values:  {df_usage.shape}')

Before combine_kwh_delivered_and_received_values: (182804, 36)
After combine_kwh_delivered_and_received_values:  (138266, 36)


In [19]:
df_usage = df_usage[df_usage['aep_derived_uom']=='KWH'].copy()
start = time.time()
df_usage, n_events_with_mult_matches = clm.match_events_in_df_to_outages(df_usage, df_outage, 
                                                                              return_n_events_with_mult_matches=True, 
                                                                              events_cols = {'premise_nb':'aep_premise_nb', 
                                                                                             'event_time':'starttimeperiod_utc'})
cols_to_return_from_df_outage = ['DT_OFF_TS_FULL', 'DT_ON_TS', 'STEP_DRTN_NB', 'search_time_window', 'LOCATION_ID', 'GIS_CRCT_NB']
placement_cols =                ['DT_OFF_TS_FULL', 'DT_ON_TS', 'STEP_DRTN_NB', 'search_time_window', 'LOCATION_ID', 'GIS_CRCT_NB']
df_usage = clm.set_all_outages_info_in_events_df(df_usage, df_outage, 
                                                     cols_to_return_from_df_outage=cols_to_return_from_df_outage, 
                                                     placement_cols=placement_cols)
time_new = time.time() - start
print(f'n_events_with_mult_matches = {n_events_with_mult_matches}')

Number of events matched with multiple outages: 0
BEFORE EXPLODE: n_rows = 44538
AFTER EXPLODE: n_rows = 44538
delta = 0
n_events_with_mult_matches = 0


In [ ]:
df_usage['outg_rec_nb']==df_usage['OUTG_REC_NB']

In [ ]:
df_usage[['outg_rec_nb', 'OUTG_REC_NB']]

In [ ]:
df_usage[df_usage['outg_rec_nb']!=df_usage['OUTG_REC_NB']].shape

In [ ]:
# Need to be a little careful with n_events_with_mult_matches below, as values were exploded in match_events_in_df_to_outages
# Therefore, naively taking df_end_dev_event_cpy[df_end_dev_event_cpy['n_outages_found'] > 1].shape[0] would overcount.
#   e.g., if every event with 'n_outages_found'>1 found two outages, the value returned by the naive method mentioned
#         above would be twice the actual value
# Not sure what solution is for this case!
#n_events_with_mult_matches = df_usage[df_usage['n_outages_found'] > 1].shape[0]
n_events_with_no_matches = df_usage[df_usage['n_outages_found'] < 1].shape[0]
#print(f'n_events_with_mult_matches = {n_events_with_mult_matches}')
print(f'n_events_with_no_matches = {n_events_with_no_matches}')

In [ ]:
#df_usage[df_usage['n_outages_found'] < 1]

In [ ]:
#df_outage[df_outage['PREMISE_NB']=='107220620']

In [ ]:
def build_usage_plots_around_time_of_outage(fig_num, df_usage, outg_rec_nb, orientation='portrait'):
    assert(orientation=='landscape' or orientation=='portrait')
    #-----
    assert(outg_rec_nb in df_usage['OUTG_REC_NB'].unique())
    df_usage_outg_i = df_usage[df_usage['OUTG_REC_NB']==outg_rec_nb]

    assert(len(df_usage_outg_i['DT_OFF_TS_FULL'].unique())==1)
    assert(len(df_usage_outg_i['DT_ON_TS'].unique())==1)
    out_beg = df_usage_outg_i.iloc[0]['DT_OFF_TS_FULL']
    out_end = df_usage_outg_i.iloc[0]['DT_ON_TS']
    #----------------------------------------------
    sns_unq = df_usage_outg_i['serialnumber'].unique().tolist()
    palette_dict = sns.color_palette('colorblind', len(sns_unq))
    palette_dict = {sns_unq[i]:palette_dict[i] for i in range(len(sns_unq))}

    linestyles_dict = Plot_General.get_standard_linestyles(len(sns_unq), exclude_loose=True)
    linestyles_dict = {sns_unq[i]:linestyles_dict[i][1] for i in range(len(sns_unq))}
    #----------------------------------------------
    cent_delt = '30 minutes'
    tmp_df_cent_pm_delt = df_usage_outg_i[(df_usage_outg_i['starttimeperiod_utc'] >= df_usage_outg_i['DT_OFF_TS_FULL']-pd.Timedelta(cent_delt)) & 
                                 (df_usage_outg_i['starttimeperiod_utc'] <= df_usage_outg_i['DT_ON_TS']+pd.Timedelta(cent_delt))]

    tmp_df_bef_w_cent = df_usage_outg_i[(df_usage_outg_i['starttimeperiod_utc'] < df_usage_outg_i['DT_OFF_TS_FULL'] + 0.5*pd.to_timedelta(df_usage_outg_i['DT_ON_TS']-df_usage_outg_i['DT_OFF_TS_FULL'])) & 
                                  (df_usage_outg_i['starttimeperiod_utc'] >= df_usage_outg_i['DT_OFF_TS_FULL'] - df_usage_outg_i['search_time_window'])]

    tmp_df_bef = df_usage_outg_i[(df_usage_outg_i['starttimeperiod_utc'] < df_usage_outg_i['DT_OFF_TS_FULL']) & 
                                 (df_usage_outg_i['starttimeperiod_utc'] >= df_usage_outg_i['DT_OFF_TS_FULL'] - df_usage_outg_i['search_time_window'])]

    tmp_df_aft_w_cent = df_usage_outg_i[(df_usage_outg_i['starttimeperiod_utc'] < df_usage_outg_i['DT_ON_TS'] + df_usage_outg_i['search_time_window']) & 
                                 (df_usage_outg_i['starttimeperiod_utc'] >= df_usage_outg_i['DT_ON_TS'] - 0.5*pd.to_timedelta(df_usage_outg_i['DT_ON_TS']-df_usage_outg_i['DT_OFF_TS_FULL']))]

    tmp_df_aft = df_usage_outg_i[(df_usage_outg_i['starttimeperiod_utc'] < df_usage_outg_i['DT_ON_TS'] + df_usage_outg_i['search_time_window']) & 
                                 (df_usage_outg_i['starttimeperiod_utc'] >= df_usage_outg_i['DT_ON_TS'])]
    #---------------------------------------------------------------------------------------------------------
    minmax_bef = [tmp_df_bef['value'].min(), tmp_df_bef['value'].max()]
    minmax_aft = [tmp_df_aft['value'].min(), tmp_df_aft['value'].max()]
    minmax = [min(minmax_bef[0], minmax_aft[0]), max(minmax_bef[1], minmax_aft[1])]
    #---------------------------------------------------------------------------------------------------------
    dfs_dicts = [{'df':df_usage_outg_i,     'title':'Full Range',                         'draw_out_beg':True,  'draw_out_end':True, 'minmax':None}, 
                 {'df':tmp_df_cent_pm_delt, 'title':r'Center $\pm$ {}'.format(cent_delt), 'draw_out_beg':True,  'draw_out_end':True, 'minmax':None}, 
                 {'df':tmp_df_bef_w_cent,   'title':'Before with Center',                 'draw_out_beg':True,  'draw_out_end':False, 'minmax':None}, 
                 {'df':tmp_df_bef,          'title':'Before',                             'draw_out_beg':True,  'draw_out_end':False, 'minmax':minmax}, 
                 {'df':tmp_df_aft_w_cent,   'title':'After with Center',                  'draw_out_beg':False, 'draw_out_end':True, 'minmax':None}, 
                 {'df':tmp_df_aft,          'title':'After',                              'draw_out_beg':False, 'draw_out_end':True, 'minmax':minmax}]
    #---------------------------------------------------------------------------------------------------------
    if orientation=='landscape':
        plots_nx = 3
        plots_ny = 2
        top = mpl.rcParams["figure.subplot.top"]
        bottom = mpl.rcParams["figure.subplot.bottom"]
        left = 0.05
        right = 0.90
        fig, axs = plt.subplots(plots_ny, plots_nx, num=fig_num, figsize=[11*plots_nx, 8.5*plots_ny])
    else:
        plots_nx = 2
        plots_ny = 3
        top = 0.925
        bottom = 0.075
        left = 0.075 
        right = 0.90
        fig, axs = plt.subplots(plots_ny, plots_nx, num=fig_num, figsize=[11*plots_nx, 8.5*plots_ny])
    assert(len(dfs_dicts)==plots_ny*plots_nx)

    plt.subplots_adjust(wspace=0.2, hspace=0.3, left=left, right=right, top=top, bottom=bottom)
    #--------------------------------------------------------------
    for i, df_dict in enumerate(dfs_dicts):
        df = df_dict['df']
        title = df_dict['title'] 
        draw_out_beg = df_dict['draw_out_beg'] 
        draw_out_end = df_dict['draw_out_end'] 
        minmax = df_dict['minmax'] 
        #---------------
        i_y, i_x = divmod(i, plots_nx)
        ax = axs[i_y, i_x]
        #---------------
        sns.lineplot(ax=ax, x='starttimeperiod_utc', y='value', hue='serialnumber', data=df, 
                     palette=palette_dict, style='serialnumber', dashes=linestyles_dict, legend=False)
        if draw_out_beg:
            ax.axvline(x=out_beg, color='red')
        if draw_out_end:
            ax.axvline(x=out_end, color='lawngreen')
        ax.set_title(title, fontsize=30)
        if minmax is not None:
            ax.set_ylim(minmax)
    #--------------------------------------------------------------
    if len(df_usage_outg_i['serialnumber'].unique()) < 50:
        leg_handles = build_sn_legend_handles(df_usage_outg_i['serialnumber'].unique(), palette_dict, linestyles_dict)
        fig.legend(title='serialnumber', handles=leg_handles, fontsize=20, title_fontsize=30, 
                   bbox_to_anchor=(1.01*right, top), loc='upper left')
    fig.supxlabel('starttimeperiod_utc', fontsize=50, 
                  x=0.9, ha='right', 
                  y=0.0, va='bottom')
    fig.supylabel('value', fontsize=50, 
                  x=0.0, ha='left',
                  y=0.9, va='top')
    fig.suptitle(f'Outage Record Number: {round(outg_rec_nb)}', fontsize=50, 
                 x=0.5*(left+right), ha='center')

    for ax in axs.flatten():
        ax.tick_params(axis='x', labelrotation=45, labelsize=15.0, direction='out');
        ax.tick_params(axis='y', labelrotation=0, labelsize=15.0, direction='out');
        ax.xaxis.label.set_size(0)
        ax.yaxis.label.set_size(0)

    return fig, axs


In [ ]:
def estimate_outage_times_for_single_meter_sn(df_single_sn, 
                                              time_col='starttimeperiod_utc', value_col='value'):
    #TODO probably delete me since I am so specialized
    # Use estimate_outage_times instead
    df_single_sn = df_single_sn.copy()

    df_single_sn = df_single_sn.reset_index()  # resets indices and creates 'index' column with original indices
    df_single_sn = df_single_sn.sort_values(by=time_col, ignore_index=True)
    found_zeros_df = df_single_sn[df_single_sn[value_col] <= 0]
    #------------------
    ids_in_groups = []
    for k, g in itertools.groupby(enumerate(found_zeros_df[value_col].index), lambda ix : ix[0] - ix[1]):
        ids_in_groups.append([x[1] for x in list(g)])
    
    outage_times = [pd.to_datetime(found_zeros_df.loc[group, time_col].unique()).tolist() for group in ids_in_groups]
    return outage_times


# def consecutive(data, stepsize=pd.Timedelta('15 minutes')):
#     return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)

def consecutive(
    data, 
    stepsize=pd.Timedelta('15 minutes')
):
    return Utilities.get_fuzzy_overlap(data, stepsize)

def estimate_outage_times(df, require_all_sns_zero_for_outage=True, 
                          time_col='starttimeperiod_utc', value_col='value', sn_col='serialnumber'):
    # Intended for use with single OUTG_REC_NB
    # BUT is designed to find multipe groups of zero readings
    df = df.copy()
    all_sns = df[sn_col].unique().tolist()
    df = df.reset_index() # resets indices and creates 'index' column with original indices
    df = df.sort_values(by=time_col, ignore_index=True)
        
    found_zeros_df = df[df[value_col] <= 0]
    if not require_all_sns_zero_for_outage:
        outage_times = pd.to_datetime(found_zeros_df[time_col].unique()).tolist()
    else:
        outage_times = []
        for time, tmp_df in found_zeros_df.groupby(time_col):
            sns = tmp_df[sn_col].unique().tolist()
            if set(sns)==set(all_sns):
                outage_times.append(time)
    return consecutive(outage_times)

In [ ]:
# tmp_df = df_usage[df_usage['OUTG_REC_NB']==11947217].copy()
# tmp_df2 = df_usage[df_usage['OUTG_REC_NB']==11947217].copy()

# tmp_sns = tmp_df2['serialnumber'].unique().tolist()

# tmp_df2.loc[(tmp_df2['starttimeperiod'] >pd.to_datetime('2020-10-12T09:00:00.000000000')) & 
#             (tmp_df2['starttimeperiod']<=pd.to_datetime('2020-10-12T11:00:00.000000000')), 'value'] = 0.0

# tmp_df2.loc[(tmp_df2['starttimeperiod']>pd.to_datetime('2020-10-12T09:00:00.000000000')) & 
#             (tmp_df2['starttimeperiod']<=pd.to_datetime('2020-10-12T12:00:00.000000000')) & 
#             (tmp_df2['serialnumber'].isin(tmp_sns[:5])), 'value'] = 0.0

# tmp_df2_cpy = tmp_df2.copy()


# fig, axs = build_usage_plots_around_time_of_outage(fig_num, tmp_df2, 11947217, orientation='portrait')
# fig_num += 1


# all_outage_times = estimate_outage_times(tmp_df2)
# all_outage_times2 = estimate_outage_times(tmp_df2, require_all_sns_zero_for_outage=False)
# all_outage_times_naive = estimate_outage_times_for_single_meter_sn(tmp_df2)
# print(all_outage_times)
# print(all_outage_times2)
# print(all_outage_times_naive)

In [ ]:
df_usage.columns

In [ ]:
temp_df = df_usage[df_usage['trsf_pole_nb']==df_usage['LOCATION_ID']].copy()
fig, axs = build_usage_plots_around_time_of_outage(fig_num, temp_df, temp_df['OUTG_REC_NB'].unique().tolist()[0], orientation='portrait')
fig_num += 1

In [ ]:
fig, axs = build_usage_plots_around_time_of_outage(fig_num, df_usage, df_usage['OUTG_REC_NB'].unique().tolist()[0], orientation='portrait')
fig_num += 1

In [ ]:
df_usage['OUTG_REC_NB'].unique()

In [ ]:
len(df_usage['serialnumber'].unique())

In [ ]:
for outg_rec_nb in df_usage['OUTG_REC_NB'].unique().tolist():
    if pd.isnull(outg_rec_nb):
        continue
    fig, axs = build_usage_plots_around_time_of_outage(fig_num, df_usage, outg_rec_nb)
    fig_num += 1

In [ ]:
len(df_usage['serialnumber'].unique())

In [ ]:
df_usage['value'].mean()

In [ ]:
df_usage['value'].std()

In [ ]:
pd.to_datetime('2020-10-12T09:00:00.000000000')+pd.to_timedelta('4h')

In [ ]:
df_usage[['starttimeperiod', 'starttimeperiod_utc']]

In [ ]:
# df_usage.loc[(df_usage['starttimeperiod'] >pd.to_datetime('2020-10-12T09:00:00.000000000')) & 
#                  (df_usage['starttimeperiod']<=pd.to_datetime('2020-10-12T14:00:00.000000000')), 'value'].mean()

df_usage.loc[(df_usage['starttimeperiod_utc'] >pd.to_datetime('2020-10-12T09:00:00.000000000')+pd.to_timedelta('4h')) & 
                 (df_usage['starttimeperiod_utc']<=pd.to_datetime('2020-10-12T14:00:00.000000000')+pd.to_timedelta('4h')), 'value'].mean()

In [ ]:
# df_usage.loc[(df_usage['starttimeperiod'] >pd.to_datetime('2020-10-12T09:00:00.000000000')) & 
#                  (df_usage['starttimeperiod']<=pd.to_datetime('2020-10-12T14:00:00.000000000')), 'value'].std() 

df_usage.loc[(df_usage['starttimeperiod_utc'] >pd.to_datetime('2020-10-12T09:00:00.000000000')+pd.to_timedelta('4h')) & 
                 (df_usage['starttimeperiod_utc']<=pd.to_datetime('2020-10-12T14:00:00.000000000')+pd.to_timedelta('4h')), 'value'].std() 

In [ ]:
# df_usage.loc[(df_usage['starttimeperiod'] >pd.to_datetime('2020-10-12T22:00:00.000000000')) & 
#                  (df_usage['starttimeperiod']<=pd.to_datetime('2020-10-13T08:00:00.000000000')), 'value'].mean()

df_usage.loc[(df_usage['starttimeperiod_utc'] >pd.to_datetime('2020-10-12T22:00:00.000000000')+pd.to_timedelta('4h')) & 
                 (df_usage['starttimeperiod_utc']<=pd.to_datetime('2020-10-13T08:00:00.000000000')+pd.to_timedelta('4h')), 'value'].mean()

In [ ]:
# df_usage.loc[(df_usage['starttimeperiod'] >pd.to_datetime('2020-10-12T22:00:00.000000000')) & 
#                  (df_usage['starttimeperiod']<=pd.to_datetime('2020-10-13T08:00:00.000000000')), 'value'].std()

df_usage.loc[(df_usage['starttimeperiod_utc'] >pd.to_datetime('2020-10-12T22:00:00.000000000')+pd.to_timedelta('4h')) & 
                 (df_usage['starttimeperiod_utc']<=pd.to_datetime('2020-10-13T08:00:00.000000000')+pd.to_timedelta('4h')), 'value'].std()

In [ ]:
df_usage[df_usage['value']>0]['value'].mean()

In [ ]:
df_usage[df_usage['value']>0]['value'].std()

In [ ]:
# fig, ax = plt.subplots(1, 1, num=fig_num, figsize=[11, 8.5])
# sns.lineplot(ax=ax, x='starttimeperiod_utc', y='value', hue='serialnumber', 
#              data=df_usage[(df_usage['starttimeperiod']>=pd.to_datetime('2020-10-12T22:00:00.000000000')) & 
#                                (df_usage['value']>140)], 
#              legend=False)
# fig_num += 1

In [ ]:
# fig, ax = plt.subplots(1, 1, num=fig_num, figsize=[11, 8.5])
# sns.lineplot(ax=ax, x='starttimeperiod_utc', y='value', hue='serialnumber', 
#              data=df_usage[(df_usage['starttimeperiod']<=pd.to_datetime('2020-10-12T16:00:00.000000000')) & 
#                                (df_usage['value']>140)], 
#              legend=False)
# fig_num += 1

In [ ]:
df_usage.sort_values(by=['serialnumber', 'starttimeperiod_utc'])

In [ ]:
df_usage.groupby('serialnumber').describe().T

In [ ]:
df_usage.groupby('serialnumber').describe()

In [ ]:
df_usage['starttimeperiod_utc'].nunique()

In [ ]:
df_usage.set_index(['OUTG_REC_NB', 'serialnumber', 'starttimeperiod_utc']).sort_index()

In [ ]:
df_usage.set_index(['OUTG_REC_NB', 'starttimeperiod_utc', 'serialnumber']).sort_index()

In [ ]:
df_usage_mi = df_usage.set_index(['OUTG_REC_NB', 'serialnumber', 'starttimeperiod_utc']).sort_index().copy()

In [ ]:
df_usage_mi

In [ ]:
# df_usage_mi.loc[(10143524.0, slice(None), '2017-09-16 06:15:00'), 'aep_premise_nb']

In [ ]:
avg_df = df_usage_mi.groupby(['OUTG_REC_NB', 'starttimeperiod_utc'])['value'].mean()

In [ ]:
# avg_df.loc[10143524.0].plot()

In [ ]:
df_outage[df_outage['OUTG_REC_NB']==11947217]

In [ ]:
df_usage[df_usage['OUTG_REC_NB']==11947217]

In [ ]:
sorted(df_usage[df_usage['OUTG_REC_NB']==11947217]['starttimeperiod_utc'].unique())